# RN-A · Métricas por coordenada (Δα⋆, Δδ) con IC-95% + P68 radial
Calcula **RMS por coordenada** (mas) con **IC-95% bootstrap (B=5000; seed=47)** y añade **CI para P68 radial**.
Actualiza `rnA_metrics.json` en `data/...` (y también en `release/...` si existe).

In [1]:
from pathlib import Path
import json
import numpy as np, pandas as pd
from IPython.display import display

ROOT = Path.home()/ 'portafolio-rubin'
PARQUET = ROOT/'data/47tuc_dp1/rnA_matched_minimal.parquet'
JSONP   = ROOT/'data/47tuc_dp1/rnA_metrics.json'
JSON_REL= ROOT/'release/rnA_v1.0/rnA_metrics.json'
B=5000; SEED=47

df = pd.read_parquet(PARQUET)
display(df.head())

,objectId,coord_ra,coord_dec,source_id,ra_gaia,dec_gaia,separation_arcsec
534,579578142646088111,5.075909,-71.855793,4689825045638616960,5.075907,-71.855792,0.000000
889,579578692401895176,5.742830,-71.701867,4689849853371562752,5.742831,-71.701865,0.005324
462,579577455451312256,4.844347,-72.044410,4689809137079932544,4.844343,-72.044410,0.005324
455,579578761121371709,5.673968,-71.731994,4689849754585727744,5.673963,-71.731994,0.006147
162,579578142646084079,5.069619,-71.928691,4689812092017502208,5.069616,-71.928690,0.006147


In [2]:
import numpy as np
deg2asec = 3600.0
dec_mid = 0.5*(df['coord_dec'].to_numpy() + df['dec_gaia'].to_numpy())
cosd = np.cos(np.deg2rad(dec_mid))
dra_arcsec  = (df['coord_ra'].to_numpy()  - df['ra_gaia'].to_numpy()) * cosd * deg2asec
ddec_arcsec = (df['coord_dec'].to_numpy() - df['dec_gaia'].to_numpy()) * deg2asec
sep_arcsec  = df['separation_arcsec'].to_numpy()
N = len(df)
N

1113

In [3]:
rng = np.random.default_rng(SEED)
def rms(x):
    x=np.asarray(x); return float(np.sqrt(np.mean(x**2)))
def ci_rms(x, B=B):
    x=np.asarray(x); n=len(x)
    idx = rng.integers(0, n, size=(B,n))
    samples = np.sqrt(np.mean((x[idx])**2, axis=1))
    lo, hi = np.quantile(samples, [0.025, 0.975])
    return float(lo), float(hi)
def ci_percentile(x, q, B=B):
    x=np.asarray(x); n=len(x)
    idx = rng.integers(0, n, size=(B,n))
    samples = np.percentile(x[idx], q, axis=1)
    lo, hi = np.quantile(samples, [0.025, 0.975])
    return float(lo), float(hi)

In [4]:
res = {
  'N_pairs': int(N),
  'per_coordinate': {
     'rms_ra_mas' : rms(dra_arcsec)*1000.0,
     'rms_dec_mas': rms(ddec_arcsec)*1000.0,
     'rms_ra_ci95_mas'  : [v*1000.0 for v in ci_rms(dra_arcsec)],
     'rms_dec_ci95_mas' : [v*1000.0 for v in ci_rms(ddec_arcsec)],
     'note': 'Δα⋆ incluye cosδ; unidades mas por coordenada.'
  },
  'radial': {
     'P50_arcsec': float(np.percentile(sep_arcsec, 50)),
     'P68_arcsec': float(np.percentile(sep_arcsec, 68)),
     'P95_arcsec': float(np.percentile(sep_arcsec, 95)),
     'P50_ci95_arcsec': list(ci_percentile(sep_arcsec, 50)),
     'P68_ci95_arcsec': list(ci_percentile(sep_arcsec, 68)),
     'P95_ci95_arcsec': list(ci_percentile(sep_arcsec, 95)),
  }
}
import pandas as pd
summary = pd.DataFrame({
  'metric'   : ['rms_ra_mas','rms_dec_mas','P50_arcsec','P68_arcsec','P95_arcsec'],
  'value'    : [res['per_coordinate']['rms_ra_mas'], res['per_coordinate']['rms_dec_mas'],
                res['radial']['P50_arcsec'],res['radial']['P68_arcsec'],res['radial']['P95_arcsec']],
  'ci95_low' : [res['per_coordinate']['rms_ra_ci95_mas'][0],res['per_coordinate']['rms_dec_ci95_mas'][0],
                res['radial']['P50_ci95_arcsec'][0],res['radial']['P68_ci95_arcsec'][0],res['radial']['P95_ci95_arcsec'][0]],
  'ci95_high': [res['per_coordinate']['rms_ra_ci95_mas'][1],res['per_coordinate']['rms_dec_ci95_mas'][1],
                res['radial']['P50_ci95_arcsec'][1],res['radial']['P68_ci95_arcsec'][1],res['radial']['P95_ci95_arcsec'][1]],
  'units'    : ['mas','mas','arcsec','arcsec','arcsec']
})
summary

,metric,value,ci95_low,ci95_high,units
0,rms_ra_mas,130.909936,97.659730,162.340758,mas
1,rms_dec_mas,146.939246,104.342334,185.351570,mas
2,P50_arcsec,0.050970,0.050691,0.051431,arcsec
3,P68_arcsec,0.053325,0.052969,0.053766,arcsec
4,P95_arcsec,0.115842,0.097805,0.170690,arcsec


In [5]:
def update_json(path):
    existing={}
    if path.exists():
        try:
            existing=json.loads(path.read_text())
        except Exception as e:
            print('WARN: no se pudo leer JSON existente:', e)
    existing['N_pairs']=res['N_pairs']
    existing['P50_arcsec']=res['radial']['P50_arcsec']
    existing['P68_arcsec']=res['radial']['P68_arcsec']
    existing['P95_arcsec']=res['radial']['P95_arcsec']
    existing['P50_ci95_arcsec']=res['radial']['P50_ci95_arcsec']
    existing['P68_ci95_arcsec']=res['radial']['P68_ci95_arcsec']
    existing['P95_ci95_arcsec']=res['radial']['P95_ci95_arcsec']
    existing['per_coordinate']=res['per_coordinate']
    existing.setdefault('units','arcsec')
    existing.setdefault('bootstrap_B', 5000)
    existing.setdefault('bootstrap_seed', 47)
    existing['generator']={'script':'notebooks/47tuc/rnA_per_coordinate_metrics.ipynb','version':'v1.1-rnA'}
    path.write_text(json.dumps(existing, indent=2))
    return existing

JSONP = Path(JSONP); JSON_REL = Path(JSON_REL)
updated = update_json(JSONP)
if JSON_REL.exists():
    update_json(JSON_REL)
from IPython.display import display
display(updated)

{'N_pairs': 1113,
 'P50_arcsec': 0.050969643129182965,
 'P68_arcsec': 0.05332470894897676,
 'P95_arcsec': 0.11584190795115097,
 'P50_ci95_arcsec': [0.050690864503642435, 0.051430916269940744],
 'P95_ci95_arcsec': [0.09780514929323501, 0.17069029791484716],
 'bootstrap_B': 5000,
 'bootstrap_seed': 47,
 'method_note': '1:1 dentro de 2″; percentiles sobre separaciones; IC-95% por bootstrap.',
 'units': 'arcsec',
 'generator': {'script': 'notebooks/47tuc/rnA_per_coordinate_metrics.ipynb',
  'version': 'v1.1-rnA'},
 'P68_ci95_arcsec': [0.05296920692603931, 0.053765781052101044],
 'per_coordinate': {'rms_ra_mas': 130.90993556974593,
  'rms_dec_mas': 146.93924562235915,
  'rms_ra_ci95_mas': [97.65973016695716, 162.34075753107206],
  'rms_dec_ci95_mas': [104.34233398890363, 185.35156993450624],
  'note': 'Δα⋆ incluye cosδ; unidades mas por coordenada.'}}